In [15]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import tqdm

In [16]:
f = np.load('/home/abdussamad/github/portfolio-manager/BackTest/inputs.npz')

In [17]:
f.files

['arr_0', 'arr_1']

In [18]:
Xs, ys = f['arr_0'], f['arr_1']

In [ ]:
plt.hist(ys)

In [ ]:
Xs[:, 0, 1].reshape(-1, 1, 1)

In [19]:
scales = Xs[:, 0, 1].reshape(-1, 1, 1).copy()
Xs[:, :, [5]] /= scales
Xs[:, :, :4] /= scales

In [ ]:
for i in range(4):
    plt.plot(Xs[399950, :, i])
plt.plot(Xs[399950, :, 5])    

In [20]:
def bresenham(img, x0, y0, x1, y1):
    dx = abs(x1 - x0)
    sx = 1 if x0 < x1 else -1 
    dy = -abs(y1 - y0)
    sy = 1 if y0 < y1 else -1
    error = dx + dy

    while True:
        img[x0, y0] = 255
        if x0 == x1 and y0 == y1:
            break
        e2 = 2 * error
        if e2 >= dy:
            if x0 == x1:
                break 
            error += dy
            x0 += sx
        if e2 <= dx:
            if y0 == y1:
                break
            error += dx
            y0 += sy

def make_image(sample, ema=True, volume=True):
    height_bars = 96
    width = sample.shape[0] * 3
    img_ohlc = np.zeros((width, height_bars), dtype=np.uint8)

    max_price = max(sample[:, :4].max(), sample[:, 5].max())
    min_price = min(sample[:, :4].min(), sample[:, 5].min())
    height_scaler = (height_bars - 1) / (max_price - min_price)

    ema_y_prev = None

    for t in range(sample.shape[0]):
        open_y = round((sample[t, 1] - min_price) * height_scaler)
        img_ohlc[3*t, open_y] = 255
        close_y = round((sample[t, 0] - min_price) * height_scaler)
        img_ohlc[3*t+2, close_y] = 255

        low_y = round((sample[t, 3] - min_price) * height_scaler)
        high_y = round((sample[t, 2] - min_price) * height_scaler)
        img_ohlc[3*t+1, low_y:high_y] = 255

        if ema:
            ema_y = round((sample[t, 5] - min_price) * height_scaler)
            img_ohlc[3*t+1, ema_y] = 255
            if ema_y_prev is not None:
                bresenham(img_ohlc, 3*t-2, ema_y_prev, 3*t+1, ema_y)
            ema_y_prev = ema_y

    if not volume:
        return img_ohlc.T
    
    height_vol = 24
    height_whole = height_bars + height_vol if volume else 0
    img_whole = np.zeros((width, height_whole), dtype=np.uint8)
    img_whole[:, :height_bars] = img_ohlc

    max_vol = sample[:, 4].max()
    vol_scaler = (height_vol - 1)/max_vol 
    for t in range(sample.shape[0]):
        vol_y = round(sample[t, 4] * vol_scaler)
        img_whole[3*t+1, height_whole-vol_y-1:height_whole-1] = 255

    return img_whole.T



In [21]:
class StockCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 64, kernel_size=(5, 3), padding=1),
            torch.nn.LeakyReLU(negative_slope=0.01, inplace=True),
            torch.nn.MaxPool2d((2, 1))
        )
        self.block2 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=(5, 3), padding=1),
            torch.nn.LeakyReLU(negative_slope=0.01, inplace=True),
            torch.nn.MaxPool2d((2, 1))
        )
        self.out_block = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(161280, 2),
            torch.nn.Softmax(dim=-1))

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.out_block(x)
        return x

In [22]:
ys_pt = torch.LongTensor(ys > 0)
xs_pt = torch.zeros(size=(Xs.shape[0], 120, 45), dtype=torch.uint8)

In [23]:
for i in range(Xs.shape[0]):
    xs_pt[i] = torch.tensor(make_image(Xs[i]))

In [24]:
split_idx = round(ys_pt.shape[0] * 0.85)
ys_train, ys_valid = ys_pt[:split_idx], ys_pt[split_idx:]
xs_train, xs_valid = xs_pt[:split_idx], xs_pt[split_idx:]

In [32]:
ds_train = torch.utils.data.TensorDataset(xs_train, ys_train)
ds_valid = torch.utils.data.TensorDataset(xs_valid, ys_valid)

BATCH_SIZE = 128
EPOCHS = 100
LEARNING_RATE = 1e-4
DEVICE = 'cuda'

m = StockCNN()
m = m.to(DEVICE)
opt = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

# ds_train_bs = torch.utils.data.TensorDataset(xs_train[:BATCH_SIZE], ys_train[:BATCH_SIZE])

dl_train = torch.utils.data.DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
dl_valid = torch.utils.data.DataLoader(ds_valid, batch_size=BATCH_SIZE, shuffle=True)

for ep in range(EPOCHS):
    train_total_loss = 0
    train_n_batches = 0
    train_n_hits = 0
    train_n_total = 0

    for xs, ys in dl_train:
        xs = xs.to(DEVICE)
        xs = xs.unsqueeze(1).float() / 255.0
        ys = ys.to(DEVICE).squeeze()
        opt.zero_grad()
        y_hat = m(xs)
        loss = loss_fn(y_hat, ys)
        loss.backward()

        train_total_loss += loss.cpu().item()
        train_n_batches += 1
        train_n_hits += (torch.argmax(y_hat, dim=-1) == ys).sum().cpu().item()
        train_n_total += y_hat.shape[0]


        # print(f'Loss={loss.cpu().item()}, {(torch.argmax(y_hat, dim=-1) == ys).sum().cpu().item()/y_hat.shape[0]}')
        opt.step()

    valid_total_loss = 0
    valid_n_batches = 0
    valid_n_hits = 0
    valid_n_total = 0

    with torch.inference_mode():
        for xs, ys in dl_valid:
            xs = xs.to(DEVICE)
            xs = xs.unsqueeze(1).float() / 255.0
            ys = ys.to(DEVICE).squeeze()
            y_hat = m(xs)
            loss = loss_fn(y_hat, ys)
            
            valid_total_loss += loss.cpu().item()
            valid_n_batches += 1
            valid_n_hits += (torch.argmax(y_hat, dim=-1) == ys).sum().cpu().item()
            valid_n_total += y_hat.shape[0]     

    print(f'Epoch {ep+1}: Train({train_total_loss/train_n_batches}, {100.0 * train_n_hits/train_n_total:.2f}), Valid({valid_total_loss/valid_n_batches}, {100.0 * valid_n_hits/valid_n_total:.2f})')       


Epoch 1: Train(0.6910760296973744, 52.82), Valid(0.6893467584519845, 53.78)
Epoch 2: Train(0.6896473602427685, 53.31), Valid(0.6895116291129466, 53.75)
Epoch 3: Train(0.6878780919489746, 54.09), Valid(0.6897134968361465, 53.62)
Epoch 4: Train(0.6854344285020784, 54.97), Valid(0.6907506457768896, 53.33)
Epoch 5: Train(0.6822858538280676, 55.95), Valid(0.6938396371291496, 52.34)
Epoch 6: Train(0.6784544911104483, 57.01), Valid(0.6946173781341177, 52.74)
Epoch 7: Train(0.6740767819824951, 58.11), Valid(0.6971454615627043, 52.51)
Epoch 8: Train(0.6693739806153439, 59.09), Valid(0.7007452581873765, 51.94)
Epoch 9: Train(0.6645216924452638, 60.08), Valid(0.7037544005725761, 51.86)
Epoch 10: Train(0.6597307378641105, 61.03), Valid(0.7064903944233659, 52.05)
Epoch 11: Train(0.6548079588245113, 61.94), Valid(0.7100279336888418, 51.64)
Epoch 12: Train(0.6501953152023892, 62.73), Valid(0.7138528955939083, 51.40)
Epoch 13: Train(0.6457489973596615, 63.46), Valid(0.7184792677024889, 51.23)
Epoch 14

KeyboardInterrupt: 

In [ ]:
print('Test')

In [ ]:
plt.imshow(xs[0, 0].cpu().numpy())

In [ ]:
ys

In [ ]:
y_hat.shape

In [ ]:
ys.shape

In [ ]:
loss_fn(y_hat, ys.squeeze())

In [ ]:
batch_pt = torch.stack([torch.FloatTensor(x)/255.0 for x in batch])
batch_pt = batch_pt.unsqueeze(dim=1)

In [ ]:
batch_pt.shape

In [ ]:

r = m(batch_pt)

In [ ]:
r.shape

In [ ]:
128 * 28 * 45

In [ ]:
batch_pt.shape

In [ ]:
(1.0579706111328149 - 0.9999994104739358) * 1655.9946819955387

In [ ]:
max_price = max(x[:, :4].max(), x[:, 5].max())
min_price = min(x[:, :4].min(), x[:, 5].min())

In [ ]:
max_price

In [ ]:
min_price

In [ ]:
x

In [ ]:
plt.imshow(make_image(x))

In [ ]:
plt.imshow(make_image(Xs[1843]))

In [ ]:
plt.imshow(make_image(Xs[18343]))

In [ ]:
price_series = np.cumprod(Xs2 + 1, axis=-1)

In [ ]:
Xs[:10]